# Loan Prediction Project: Neural Network Model

Some basic notes about the NN we are creating here
 - We are splitting the train and test set 80:20
 - This is a with a 22x3 NN with a learning rate of 5

## Libraries used

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score

dataset = pd.read_csv("train_u6lujuX_CVtuZ9i.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Co

## Data preprocessing

In [2]:
# removing all the datapoints that have null values
nan_value = float("NaN")
dataset.replace('', nan_value, inplace=True)
dataset.dropna(inplace=True)
print("Dimensions of the dataset : ", dataset.shape)

# one-hot encoding all the categorical values
g = dataset.select_dtypes(include=['object'])
g.drop(['Loan_ID', 'Loan_Status'], axis=1, inplace=True)
print(g.head())


onehot_encoder = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame()

for feature in g.columns.values:
    print(feature)
    temp_df = pd.DataFrame(onehot_encoder.fit_transform(g[[feature]]).toarray())
    print(temp_df)
    left = pd.DataFrame(enc_df)
    right = pd.DataFrame(temp_df)

    # Join one-hot encoded DataFrame with total DataFrame of encoded values
    enc_df = pd.concat([left, right], axis=1, ignore_index=True)
      
enc_df.head()

Dimensions of the dataset :  (480, 13)
  Gender Married Dependents     Education Self_Employed Property_Area
1   Male     Yes          1      Graduate            No         Rural
2   Male     Yes          0      Graduate           Yes         Urban
3   Male     Yes          0  Not Graduate            No         Urban
4   Male      No          0      Graduate            No         Urban
5   Male     Yes          2      Graduate           Yes         Urban
Gender
       0    1
0    0.0  1.0
1    0.0  1.0
2    0.0  1.0
3    0.0  1.0
4    0.0  1.0
..   ...  ...
475  1.0  0.0
476  0.0  1.0
477  0.0  1.0
478  0.0  1.0
479  1.0  0.0

[480 rows x 2 columns]
Married
       0    1
0    0.0  1.0
1    0.0  1.0
2    0.0  1.0
3    1.0  0.0
4    0.0  1.0
..   ...  ...
475  1.0  0.0
476  0.0  1.0
477  0.0  1.0
478  0.0  1.0
479  1.0  0.0

[480 rows x 2 columns]
Dependents
       0    1    2    3
0    0.0  1.0  0.0  0.0
1    1.0  0.0  0.0  0.0
2    1.0  0.0  0.0  0.0
3    1.0  0.0  0.0  0.0
4    0.0  0

C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Dimensions of the dataset :  (480, 13)
  Gender Married Dependents     Education Self_Employed Property_Area
1   Male     Yes          1      Graduate            No         Rural
2   Male     Yes          0      Graduate           Yes         Urban
3   Male     Yes          0  Not Graduate            No         Urban
4   Male      No          0      Graduate            No         Urban
5   Male     Yes          2      Graduate           Yes         Urban
Gender
       0    1
0    0.0  1.0
1    0.0  1.0
2    0.0  1.0
3    0.0  1.0
4    0.0  1.0
..   ...  ...
475  1.0  0.0
476  0.0  1.0
477  0.0  1.0
478  0.0  1.0
479  1.0  0.0

[480 rows x 2 columns]
Married
       0    1
0    0.0  1.0
1    0.0  1.0
2    0.0  1.0
3    1.0  0.0
4    0.0  1.0
..   ...  ...
475  1.0  0.0
476  0.0  1.0
477  0.0  1.0
478  0.0  1.0
479  1.0  0.0

[480 rows x 2 columns]
Dependents
       0    1    2    3
0    0.0  1.0  0.0  0.0
1    1.0  0.0  0.0  0.0
2    1.0  0.0  0.0  0.0
3    1.0  0.0  0.0  0.0
4    0.0  0

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [3]:
#test label encoding of feature variables
label_encoder = LabelEncoder()

label_encoded = g

for column in g:
    print(column)
    label_encoded[column] = label_encoder.fit_transform(g[column])

label_encoded.shape

Gender
Gender
Married
Dependents
Education
Self_Employed
Property_Area
Married
Dependents
Education
Self_Employed
Property_Area


<ipython-input-3-078f36f6f402>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_encoded[column] = label_encoder.fit_transform(g[column])
<ipython-input-3-078f36f6f402>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_encoded[column] = label_encoder.fit_transform(g[column])


(480, 6)

(480, 6)

## Neural Network

In [4]:
X = dataset.drop(columns=['Loan_Status'])
Y = pd.get_dummies(dataset['Loan_Status'])

In [5]:
#get numerical features and scale them
X_nums = dataset.select_dtypes(exclude=['object'])
minmax_scaler = MinMaxScaler()

X_nums_scaled = minmax_scaler.fit_transform(X_nums)

print(X_nums.shape)
print(X_nums_scaled.shape)
print(X_nums_scaled[0])

(480, 5)
(480, 5)
[0.05482993 0.0445666  0.20135364 0.72972973 1.        ]
(480, 5)
(480, 5)
[0.05482993 0.0445666  0.20135364 0.72972973 1.        ]


In [6]:
left = pd.DataFrame(enc_df)
right = pd.DataFrame(X_nums_scaled)

# Join categorical and numerical Data Frames
X_scaled = pd.concat([left, right], axis=1, ignore_index=True)

print(X_scaled.shape)
print(X_scaled.iloc[[0]])

# Split and shuffle data 80:20
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, 
                                                    test_size=0.2, random_state=21)

print(X_train.shape)
print(y_test.shape)
print(X_test)
print(y_test)

(480, 20)
    0    1    2    3    4    5    6    7    8    9    10   11   12   13   14  \
0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  0.0   

        15        16        17       18   19  
0  0.05483  0.044567  0.201354  0.72973  1.0  
(384, 20)
(96, 2)
      0    1    2    3    4    5    6    7    8    9    10   11   12   13  \
96   0.0  1.0  1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0   
16   0.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0   
52   0.0  1.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0   
91   1.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0   
215  0.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
23   0.0  1.0  1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0   
198  0.0  1.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0   
149  0.0  1.0  0.0  1.0 

In [25]:
# fit to MLP classifier
mlp = MLPClassifier(solver='sgd', random_state=1, activation='relu', alpha=1e-3,verbose=5,
                   learning_rate_init=0.1, batch_size = 10, hidden_layer_sizes=(50, 10))
mlp.fit(X_train, y_train)
predictions = mlp.predict(X_test)

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

Iteration 1, loss = 1.17424886
Iteration 2, loss = 1.07781402
Iteration 3, loss = 1.10174717
Iteration 4, loss = 1.01907244
Iteration 5, loss = 1.00164091
Iteration 6, loss = 0.97198560
Iteration 7, loss = 0.96263884
Iteration 8, loss = 0.97221763
Iteration 9, loss = 0.96848949
Iteration 10, loss = 0.97881406
Iteration 11, loss = 0.96311410
Iteration 12, loss = 0.95861804
Iteration 13, loss = 0.94502759
Iteration 14, loss = 0.95816767
Iteration 1, loss = 1.17424886
Iteration 2, loss = 1.07781402
Iteration 3, loss = 1.10174717
Iteration 4, loss = 1.01907244
Iteration 5, loss = 1.00164091
Iteration 6, loss = 0.97198560
Iteration 7, loss = 0.96263884
Iteration 8, loss = 0.97221763
Iteration 9, loss = 0.96848949
Iteration 10, loss = 0.97881406
Iteration 11, loss = 0.96311410
Iteration 12, loss = 0.95861804
Iteration 13, loss = 0.94502759
Iteration 14, loss = 0.95816767
Iteration 15, loss = 0.94206556
Iteration 16, loss = 1.01332500
Iteration 17, loss = 1.10572286
Iteration 18, loss = 1.031

In [26]:
print("Accuracy : ", accuracy_score(y_test, predictions))
print("Mean Square Error : ", mean_squared_error(y_test, predictions))

print("Confusion Matrix for each label : ")
print(multilabel_confusion_matrix(y_test, predictions))

print("Classification Report : ")
print(classification_report(y_test, predictions))

Accuracy :  0.7708333333333334
Mean Square Error :  0.22916666666666666
Confusion Matrix for each label : 
[[[61  0]
  [22 13]]

 [[13 22]
  [ 0 61]]]
Classification Report : 
              precision    recall  f1-score   support

           0       1.00      0.37      0.54        35
           1       0.73      1.00      0.85        61

   micro avg       0.77      0.77      0.77        96
   macro avg       0.87      0.69      0.69        96
weighted avg       0.83      0.77      0.74        96
 samples avg       0.77      0.77      0.77        96

Accuracy :  0.7708333333333334
Mean Square Error :  0.22916666666666666
Confusion Matrix for each label : 
[[[61  0]
  [22 13]]

 [[13 22]
  [ 0 61]]]
Classification Report : 
              precision    recall  f1-score   support

           0       1.00      0.37      0.54        35
           1       0.73      1.00      0.85        61

   micro avg       0.77      0.77      0.77        96
   macro avg       0.87      0.69      0.69      

In [27]:
# Grid Search to optimize algorithm

max_iter_test = [10, 50, 100]
hidden_layer_sizes_test = [(a, b) for a in 20 * np.arange(1,5) for b in 5 * np.arange(1, 5)]

learning_rates = 0.05 * np.arange(1,5)

param_grid_test = dict(learning_rate_init=learning_rates, hidden_layer_sizes=hidden_layer_sizes_test,
                 max_iter=max_iter_test)

grid = GridSearchCV(estimator=mlp, param_grid=param_grid_test)

grid.fit(X_scaled, Y)

Iteration 1, loss = 1.18969139
Iteration 2, loss = 1.07212750
Iteration 3, loss = 1.05769057
Iteration 4, loss = 1.04210089
Iteration 5, loss = 1.02931833
Iteration 6, loss = 1.01038750
Iteration 7, loss = 1.03398649
Iteration 8, loss = 0.97176107
Iteration 9, loss = 0.97758954
Iteration 10, loss = 0.98898277
Iteration 1, loss = 1.17885659
Iteration 2, loss = 1.03488548
Iteration 3, loss = 0.99351976
Iteration 1, loss = 1.18969139
Iteration 2, loss = 1.07212750
Iteration 3, loss = 1.05769057
Iteration 4, loss = 1.04210089
Iteration 5, loss = 1.02931833
Iteration 6, loss = 1.01038750
Iteration 7, loss = 1.03398649
Iteration 8, loss = 0.97176107
Iteration 9, loss = 0.97758954
Iteration 10, loss = 0.98898277
Iteration 1, loss = 1.17885659
Iteration 2, loss = 1.03488548
Iteration 3, loss = 0.99351976


C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't 

Iteration 4, loss = 0.95255299
Iteration 5, loss = 0.98231760
Iteration 6, loss = 0.96283292
Iteration 7, loss = 0.94911123
Iteration 8, loss = 0.94131168
Iteration 9, loss = 0.90670871
Iteration 10, loss = 0.98228073
Iteration 1, loss = 1.21795017
Iteration 2, loss = 1.11559580
Iteration 3, loss = 1.11065634
Iteration 4, loss = 1.02435534
Iteration 4, loss = 0.95255299
Iteration 5, loss = 0.98231760
Iteration 6, loss = 0.96283292
Iteration 7, loss = 0.94911123
Iteration 8, loss = 0.94131168
Iteration 9, loss = 0.90670871
Iteration 10, loss = 0.98228073
Iteration 1, loss = 1.21795017
Iteration 2, loss = 1.11559580
Iteration 3, loss = 1.11065634
Iteration 4, loss = 1.02435534
Iteration 5, loss = 1.05157405
Iteration 6, loss = 0.98610799
Iteration 7, loss = 1.02163180
Iteration 8, loss = 0.98010924
Iteration 9, loss = 0.99318885
Iteration 10, loss = 1.02518563
Iteration 1, loss = 1.23187307
Iteration 2, loss = 1.16604844
Iteration 3, loss = 1.13494182
Iteration 4, loss = 1.03866448
Itera

C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't 

Iteration 6, loss = 1.03832327
Iteration 7, loss = 1.07301649
Iteration 8, loss = 1.00711858
Iteration 9, loss = 1.00311675
Iteration 10, loss = 1.02428221
Iteration 1, loss = 1.20688296
Iteration 2, loss = 1.13611776
Iteration 3, loss = 1.15501889
Iteration 4, loss = 1.05697831
Iteration 5, loss = 1.06387011
Iteration 6, loss = 1.02253760
Iteration 7, loss = 1.01685988
Iteration 6, loss = 1.03832327
Iteration 7, loss = 1.07301649
Iteration 8, loss = 1.00711858
Iteration 9, loss = 1.00311675
Iteration 10, loss = 1.02428221
Iteration 1, loss = 1.20688296
Iteration 2, loss = 1.13611776
Iteration 3, loss = 1.15501889
Iteration 4, loss = 1.05697831
Iteration 5, loss = 1.06387011
Iteration 6, loss = 1.02253760
Iteration 7, loss = 1.01685988
Iteration 8, loss = 0.97831766
Iteration 9, loss = 0.99406575
Iteration 10, loss = 0.98539974
Iteration 1, loss = 1.18969139
Iteration 2, loss = 1.07212750
Iteration 3, loss = 1.05769057
Iteration 4, loss = 1.04210089
Iteration 5, loss = 1.02931833
Itera

...

C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 1.22201759
Iteration 2, loss = 1.01551550
Iteration 3, loss = 0.97951400
Iteration 4, loss = 0.95913423
Iteration 5, loss = 1.00286318
Iteration 6, loss = 0.97285807
Iteration 7, loss = 0.94771123
Iteration 8, loss = 0.97168184
Iteration 9, loss = 0.94961696
Iteration 10, loss = 0.96316598


GridSearchCV(estimator=MLPClassifier(alpha=0.001, batch_size=10,
                                     hidden_layer_sizes=(50, 10),
                                     learning_rate_init=0.1, random_state=1,
                                     solver='sgd', verbose=5),
             param_grid={'hidden_layer_sizes': [(20, 5), (20, 10), (20, 15),
                                                (20, 20), (40, 5), (40, 10),
                                                (40, 15), (40, 20), (60, 5),
                                                (60, 10), (60, 15), (60, 20),
                                                (80, 5), (80, 10), (80, 15),
                                                (80, 20)],
                         'learning_rate_init': array([0.05, 0.1 , 0.15, 0.2 ]),
                         'max_iter': [10, 50, 100]})

C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(estimator=MLPClassifier(alpha=0.001, batch_size=10,
                                     hidden_layer_sizes=(50, 10),
                                     learning_rate_init=0.1, random_state=1,
                                     solver='sgd', verbose=5),
             param_grid={'hidden_layer_sizes': [(20, 5), (20, 10), (20, 15),
                                                (20, 20), (40, 5), (40, 10),
                                                (40, 15), (40, 20), (60, 5),
                                                (60, 10), (60, 15), (60, 20),
                                                (80, 5), (80, 10), (80, 15),
                                                (80, 20)],
                         'learning_rate_init': array([0.05, 0.1 , 0.15, 0.2 ]),
                         'max_iter': [10, 50, 100]})

In [28]:
print("Optimal Hyper-parameters : ", grid.best_params_)
print("Optimal Accuracy : ", grid.best_score_)

Optimal Hyper-parameters :  {'hidden_layer_sizes': (60, 10), 'learning_rate_init': 0.05, 'max_iter': 10}
Optimal Accuracy :  0.8083333333333332
Optimal Hyper-parameters :  {'hidden_layer_sizes': (60, 10), 'learning_rate_init': 0.05, 'max_iter': 10}
Optimal Accuracy :  0.8083333333333332


In [29]:
# Test different Activation Functions

mlp_sigmoid = MLPClassifier(solver='sgd', random_state=1, activation='tanh', alpha=1e-3,verbose=False,
                   learning_rate_init=0.05, batch_size = 10, hidden_layer_sizes=(60, 10), max_iter=10)

mlp_relu = MLPClassifier(solver='sgd', random_state=1, activation='relu', alpha=1e-3,verbose=False,
                   learning_rate_init=0.05, batch_size = 10, hidden_layer_sizes=(60, 10), max_iter=10)

mlp_sigmoid.fit(X_train, y_train)
predictions_sig = mlp_sigmoid.predict(X_test)

mlp_relu.fit(X_train, y_train)
predictions_relu = mlp_relu.predict(X_test)

print("Neural Network Tanh: ")
print("=" * 30)
print("Training set score: %f" % mlp_sigmoid.score(X_train, y_train))
print("Test set score: %f" % mlp_sigmoid.score(X_test, y_test))

print("Accuracy : ", accuracy_score(y_test, predictions_sig))
print("Mean Square Error : ", mean_squared_error(y_test, predictions_sig))

print("Confusion Matrix for each label : ")
print(multilabel_confusion_matrix(y_test, predictions_sig))

print("Classification Report : ")
print(classification_report(y_test, predictions_sig))

print("Neural Network ReLU: ")
print("=" * 30)
print("Training set score: %f" % mlp_relu.score(X_train, y_train))
print("Test set score: %f" % mlp_relu.score(X_test, y_test))

print("Accuracy : ", accuracy_score(y_test, predictions_relu))
print("Mean Square Error : ", mean_squared_error(y_test, predictions_relu))

print("Confusion Matrix for each label : ")
print(multilabel_confusion_matrix(y_test, predictions_relu))

print("Classification Report : ")
print(classification_report(y_test, predictions_relu))

C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Xiao Bao Bao\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't 

Neural Network Tanh: 
Training set score: 0.773438
Test set score: 0.739583
Accuracy :  0.7395833333333334
Mean Square Error :  0.234375
Confusion Matrix for each label : 
[[[57  4]
  [20 15]]

 [[14 21]
  [ 0 61]]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.79      0.43      0.56        35
           1       0.74      1.00      0.85        61

   micro avg       0.75      0.79      0.77        96
   macro avg       0.77      0.71      0.70        96
weighted avg       0.76      0.79      0.74        96
 samples avg       0.77      0.79      0.77        96

Neural Network ReLU: 
Training set score: 0.817708
Test set score: 0.781250
Accuracy :  0.78125
Mean Square Error :  0.21875
Confusion Matrix for each label : 
[[[61  0]
  [21 14]]

 [[14 21]
  [ 0 61]]]
Classification Report : 
              precision    recall  f1-score   support

           0       1.00      0.40      0.57        35
           1       0.74      1.00      0